<a href="https://colab.research.google.com/github/kalleo1996/pysparkMLExample/blob/main/Copy_of_Copy_of_Copy_of_Kalpani_predict_randomforestclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 63kB/s 
     |████████████████████████████████| 204kB 12.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=38e99673856d2dbd6973f5071e54217cf31d37fa3b28e88f95f6d14fd3aa1de9
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession \
    .builder \
    .config("spark.driver.memory", "15g") \
    .appName("Exploratory Analysis") \
    .getOrCreate()

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
parking2017 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/content/gdrive/My Drive/Big Data/Parking_Violations_Issued_-_Fiscal_Year_2017.csv')
parking2016 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/content/gdrive/My Drive/Big Data/Parking_Violations_Issued_-_Fiscal_Year_2016.csv')
parking2015 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/content/gdrive/My Drive/Big Data/Parking_Violations_Issued_-_Fiscal_Year_2015.csv')
parking2014 = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load('/content/gdrive/My Drive/Big Data/Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv')

In [ ]:
parking2017 = parking2017.dropDuplicates()
parking2016 = parking2016.dropDuplicates()
parking2015 = parking2015.dropDuplicates()
parking2014 = parking2014.dropDuplicates()

In [ ]:
parking2017 = parking2017.dropna()
parking2016 = parking2016.dropna()
parking2015 = parking2015.dropna()
parking2014 = parking2014.dropna()

In [ ]:
parking2017 = parking2017.toDF(*(c.replace(' ', '_') for c in parking2017.columns))
parking2016 = parking2016.toDF(*(c.replace(' ', '_') for c in parking2016.columns))
parking2015 = parking2015.toDF(*(c.replace(' ', '_') for c in parking2015.columns))
parking2014 = parking2014.toDF(*(c.replace(' ', '_') for c in parking2014.columns))

# Predictions

In [ ]:
from pyspark.sql.functions import unix_timestamp, from_unixtime

parking2017.createOrReplaceTempView("parkingtable2017")
parking2016.createOrReplaceTempView("parkingtable2016")
parking2015.createOrReplaceTempView("parkingtable2015")
parking2014.createOrReplaceTempView("parkingtable2014")

In [ ]:
#parking2017.show(100)

In [ ]:
def get_weekday(date):
    import datetime
    import calendar
    month, day, year = (int(x) for x in date.split('/'))    
    weekday = datetime.date(year, month, day)
    #return calendar.day_name[weekday.weekday()]
    
    if calendar.day_name[weekday.weekday()] == 'Sunday':
      return 0
    if calendar.day_name[weekday.weekday()] == 'Monday':
      return 1
    if calendar.day_name[weekday.weekday()] == 'Tuesday':
      return 2
    if calendar.day_name[weekday.weekday()] == 'Wednesday':
      return 3
    if calendar.day_name[weekday.weekday()] == 'Thursday':
      return 4
    if calendar.day_name[weekday.weekday()] == 'Friday':
      return 5
    if calendar.day_name[weekday.weekday()] == 'Saturday':
      return 6
    if calendar.day_name[weekday.weekday()] == 'Sunday':
      return 7

spark.udf.register('get_weekday', get_weekday)

<function __main__.get_weekday>

In [ ]:
from pyspark.sql.functions import udf

def week_number(date_str):
    start_day_of_week=7
   
    def _week_number(date_str):
        from datetime import datetime, date
        d = datetime.strptime(date_str, '%m/%d/%Y')     # compatible with Python2
        wd_d1 = date(d.year, d.month, 1).isoweekday()
        offset = (wd_d1 - start_day_of_week + 7 ) % 7
        return (d.day - 1 + offset) // 7 + 1
    return _week_number(date_str)
spark.udf.register('week_number',week_number)    

<function __main__.week_number>

In [ ]:
from pyspark.sql.functions import udf,desc
from datetime import datetime

In [ ]:


from pyspark.sql.functions import date_format
from pyspark.sql.functions import to_date, date_format

from pyspark.sql.functions import *


In [ ]:
data_2017 = spark.sql("select year(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as year,month(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as month,week_number(Issue_Date) as week,get_weekday(Issue_Date) as day,Violation_Location, Violation_Code , count(*) as frequency from parkingtable2017  Group by year,month,week,day,Violation_Location, Violation_Code order by year,month,week,day,Violation_Location,Violation_Code ")

In [ ]:
data_2017.dtypes

[('year', 'int'),
 ('month', 'int'),
 ('week', 'string'),
 ('day', 'string'),
 ('Violation_Location', 'int'),
 ('Violation_Code', 'int'),
 ('frequency', 'bigint')]

In [ ]:
data_2017.createOrReplaceTempView("data_2017")


In [ ]:
data_2017 = spark.sql(" SELECT CAST(year AS INTEGER),CAST(month AS INTEGER),CAST(week AS INTEGER) , CAST(day AS INTEGER), CAST(Violation_Location AS INTEGER),CAST(Violation_Code AS INTEGER),frequency FROM data_2017")

In [ ]:
data_2017.dtypes

[('year', 'int'),
 ('month', 'int'),
 ('week', 'int'),
 ('day', 'int'),
 ('Violation_Location', 'int'),
 ('Violation_Code', 'int'),
 ('frequency', 'bigint')]

In [ ]:
data_2017_day = spark.sql("select year(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as year,month(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as month,week_number(Issue_Date) as week,get_weekday(Issue_Date) as day,Violation_Location,count(*) as totalViolation from parkingtable2017  Group by year,month,week,day,Violation_Location order by year,month,week,day,Violation_Location ")

In [ ]:
data_2017_day.createOrReplaceTempView("data_2017_d")

In [ ]:
data_2017_day=spark.sql(" SELECT CAST(year AS INTEGER),CAST(month AS INTEGER),CAST(week AS INTEGER) , CAST(day AS INTEGER), CAST(Violation_Location AS INTEGER),totalViolation FROM data_2017_d")

In [ ]:
data_2017_day.dtypes

[('year', 'int'),
 ('month', 'int'),
 ('week', 'int'),
 ('day', 'int'),
 ('Violation_Location', 'int'),
 ('totalViolation', 'bigint')]

In [ ]:
#data_2017_day.show(100)

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.functions import round, col
df = data_2017.join(data_2017_day, (data_2017.Violation_Location == data_2017_day.Violation_Location) & (data_2017.year == data_2017_day.year)& (data_2017.month == data_2017_day.month)& (data_2017.week == data_2017_day.week) & (data_2017.day == data_2017_day.day))\
    .withColumn("violation_probability", round(F.col("frequency") / F.col("totalViolation"), 5))\
    .drop(data_2017_day.totalViolation).drop(data_2017.frequency).drop(data_2017_day.Violation_Location).drop(data_2017_day.month).drop(data_2017_day.week).drop(data_2017_day.day).drop(data_2017_day.year)
#df.show(100)

for 2014 to 2016 first

In [ ]:
data_2014 = spark.sql("select year(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as year,month(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as month,week_number(Issue_Date) as week,get_weekday(Issue_Date) as day,Violation_Location, Violation_Code , count(*) as frequency from parkingtable2014 where   Group by year,month,week,day,Violation_Location, Violation_Code order by year,month,week,day,Violation_Location,Violation_Code ")
data_2014.createOrReplaceTempView("data_2014")
data_2014 = spark.sql(" SELECT CAST(year AS INTEGER),CAST(month AS INTEGER),CAST(week AS INTEGER) , CAST(day AS INTEGER), CAST(Violation_Location AS INTEGER),CAST(Violation_Code AS INTEGER),frequency FROM data_2014 where year>= 2014 ")

data_2015 = spark.sql("select year(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as year,month(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as month,week_number(Issue_Date) as week,get_weekday(Issue_Date) as day,Violation_Location, Violation_Code , count(*) as frequency from parkingtable2015  Group by year,month,week,day,Violation_Location, Violation_Code order by year,month,week,day,Violation_Location,Violation_Code ")
data_2015.createOrReplaceTempView("data_2015")
data_2015 = spark.sql(" SELECT CAST(year AS INTEGER),CAST(month AS INTEGER),CAST(week AS INTEGER) , CAST(day AS INTEGER), CAST(Violation_Location AS INTEGER),CAST(Violation_Code AS INTEGER),frequency FROM data_2015")


data_2016 = spark.sql("select year(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as year,month(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as month,week_number(Issue_Date) as week,get_weekday(Issue_Date) as day,Violation_Location, Violation_Code , count(*) as frequency from parkingtable2016  Group by year,month,week,day,Violation_Location, Violation_Code order by year,month,week,day,Violation_Location,Violation_Code ")
data_2016.createOrReplaceTempView("data_2016")
data_2016 = spark.sql(" SELECT CAST(year AS INTEGER),CAST(month AS INTEGER),CAST(week AS INTEGER) , CAST(day AS INTEGER), CAST(Violation_Location AS INTEGER),CAST(Violation_Code AS INTEGER),frequency FROM data_2016")



In [ ]:
data_2014.dtypes

In [ ]:
data_2014_day = spark.sql("select year(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as year,month(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as month,week_number(Issue_Date) as week,get_weekday(Issue_Date) as day,Violation_Location,count(*) as totalViolation from parkingtable2014  Group by year,month,week,day,Violation_Location order by year,month,week,day,Violation_Location ")
data_2014_day.createOrReplaceTempView("data_2014_d")
data_2014_day=spark.sql(" SELECT CAST(year AS INTEGER),CAST(month AS INTEGER),CAST(week AS INTEGER) , CAST(day AS INTEGER), CAST(Violation_Location AS INTEGER),totalViolation FROM data_2014_d where year >= 2014")

data_2015_day = spark.sql("select year(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as year,month(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as month,week_number(Issue_Date) as week,get_weekday(Issue_Date) as day,Violation_Location,count(*) as totalViolation from parkingtable2015  Group by year,month,week,day,Violation_Location order by year,month,week,day,Violation_Location ")
data_2015_day.createOrReplaceTempView("data_2015_d")
data_2015_day=spark.sql(" SELECT CAST(year AS INTEGER),CAST(month AS INTEGER),CAST(week AS INTEGER) , CAST(day AS INTEGER), CAST(Violation_Location AS INTEGER),totalViolation FROM data_2015_d")

data_2016_day = spark.sql("select year(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as year,month(from_unixtime(unix_timestamp(Issue_Date, 'MM/dd/yyy'))) as month,week_number(Issue_Date) as week,get_weekday(Issue_Date) as day,Violation_Location,count(*) as totalViolation from parkingtable2016  Group by year,month,week,day,Violation_Location order by year,month,week,day,Violation_Location ")
data_2016_day.createOrReplaceTempView("data_2016_d")
data_2016_day=spark.sql(" SELECT CAST(year AS INTEGER),CAST(month AS INTEGER),CAST(week AS INTEGER) , CAST(day AS INTEGER), CAST(Violation_Location AS INTEGER),totalViolation FROM data_2016_d")

In [ ]:
data_2014_day.dtypes

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.functions import round, col
df_2014 = data_2014.join(data_2014_day, (data_2014.Violation_Location == data_2014_day.Violation_Location) & (data_2014.year == data_2014_day.year)& (data_2014.month == data_2014_day.month)& (data_2014.week == data_2014_day.week) & (data_2014_day.day == data_2014_day.day))\
    .withColumn("violation_probability", round(F.col("frequency") / F.col("totalViolation"), 5))\
    .drop(data_2014_day.totalViolation).drop(data_2014.frequency).drop(data_2014_day.Violation_Location).drop(data_2014_day.month).drop(data_2014_day.week).drop(data_2014_day.day).drop(data_2014_day.year)

df_2015 = data_2015.join(data_2015_day, (data_2015.Violation_Location == data_2015_day.Violation_Location) & (data_2015.year == data_2015_day.year)& (data_2015.month == data_2015_day.month)& (data_2015.week == data_2015_day.week) & (data_2015_day.day == data_2015_day.day))\
    .withColumn("violation_probability", round(F.col("frequency") / F.col("totalViolation"), 5))\
    .drop(data_2015_day.totalViolation).drop(data_2015.frequency).drop(data_2015_day.Violation_Location).drop(data_2015_day.month).drop(data_2015_day.week).drop(data_2015_day.day).drop(data_2015_day.year)

df_2016 = data_2016.join(data_2016_day, (data_2016.Violation_Location == data_2016_day.Violation_Location) & (data_2016.year == data_2016_day.year)& (data_2016.month == data_2016_day.month)& (data_2016.week == data_2016_day.week) & (data_2016_day.day == data_2016_day.day))\
    .withColumn("violation_probability", round(F.col("frequency") / F.col("totalViolation"), 5))\
    .drop(data_2016_day.totalViolation).drop(data_2016.frequency).drop(data_2016_day.Violation_Location).drop(data_2016_day.month).drop(data_2016_day.week).drop(data_2016_day.day).drop(data_2016_day.year)

#df_2015.show()

In [ ]:
#dataframe concatenation
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

#def unionAll(*dfs):
    #return reduce(DataFrame.unionAll, dfs)

#full_df = unionAll( df_2015, df_2016,df)
#train_df.show(10)

In [ ]:
full_df = df.union(df_2015).union(df_2016).union(df_2014)


NameError: ignored

In [ ]:
#from pyspark.ml.feature import StringIndexer
#qualification_indexer = StringIndexer(inputCol="day", outputCol="day_chng")
#Fits a model to the input dataset with optional parameters.
#df_buck = qualification_indexer.fit(full_df).transform(full_df)
#df_buck.show()

In [ ]:
from pyspark.ml.feature import OneHotEncoder

#for training data
#

#day_encoded = OneHotEncoder(inputCol="day", outputCol="day_one")
#df_buck = day_encoded.fit(full_df).transform(full_df)

#week_encoded = OneHotEncoder(inputCol="week", outputCol="week_one")
#df_buck = week_encoded.fit(df_buck).transform(df_buck)

month_encoded = OneHotEncoder(inputCol="month", outputCol="month_one")
df_buck = month_encoded.fit(full_df).transform(full_df)

location_encoded = OneHotEncoder(inputCol="Violation_Location", outputCol="Violation_Location_one")
df_buck1 = location_encoded.fit(df_buck).transform(df_buck)

code_encoded = OneHotEncoder(inputCol="Violation_Code", outputCol="Violation_Code_one")
df_buck2 = code_encoded.fit(df_buck1).transform(df_buck1)


#df_buck_train.show(10)

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

#for training data
#assembler = VectorAssembler(inputCols=["day_one","month_one","Violation_Location_one","week_one","Violation_Code_one"], outputCol="features")
# Assemble all the features with VectorAssembler
required_features = ["year","month_one","week","day","Violation_Location_one","Violation_Code_one"]

from pyspark.ml.feature import VectorAssembler

numericCols = [] #['prob']
assemblerInputs = [c  for c in required_features] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol='features')

transformed_data = assembler.transform(df_buck2)
#transformed_data.show(5)

In [ ]:
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2], seed =2020)

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='violation_probability')
lr_model = lr.fit(training_data)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))


Coefficients: [-0.014050739142411983,0.0,0.07387566576051557,0.03433812606846102,-0.03433086883054008,-0.024713940170639062,-0.01478885755643234,-0.025192982995659464,-0.01026425733246481,-0.04388804569914226,-0.006546651999853586,-0.03936833516020695,-0.005039024228701165,0.010508538613997398,-0.0010711163446267182,0.0,-0.35770741024798003,0.0,0.0,0.0,-0.3661423481260385,-0.24868049814794918,-0.3514731076316914,0.0,-0.3153353960841353,-0.359276646583317,0.0,0.0,-0.33929306089385336,-0.32294420759323234,0.0,0.0,-0.33834310523548705,-0.3583959707397877,-0.3613781769883482,-0.3676263923845598,0.0,0.14549334281331336,-0.32433116777418347,-0.33538767969961947,-0.30133403724711805,-0.2659289219469336,0.0,-0.3468224270936365,0.0,-0.3055198819305495,0.0,-0.329082325007672,-0.29085932974654927,-0.3287834206564618,0.0,0.0,0.0,0.0,0.0,-0.18292776870815572,-0.25870226893789283,-0.2381169164819354,-0.3299920713611229,-0.3560849062298469,-0.3191641745135557,-0.35586301790178543,-0.3211884226120251,

In [ ]:
model_path = "/content/gdrive/My Drive/Big Data/" + "modelday2path"
lr_model.save(model_path)

NameError: ignored

In [ ]:
from pyspark.ml.regression import LinearRegressionModel
model2 = LinearRegressionModel.load(model_path)

In [ ]:
model2_predict = model2.transform(test_data.limit(10))

In [ ]:
model2_predict.select("prediction","violation_probability","features").show(10)

In [ ]:

print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

In [ ]:
lr_predictions = lr_model.transform(test_data)
lr_predictions.select("prediction","violation_probability","features").show(100)

In [ ]:
dataForGrapgh = lr_predictions.select("prediction","violation_probability","features")

In [ ]:
dataG =dataForGrapgh.toPandas()

In [ ]:
# Necessary imports: 
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics
accuracy = metrics.r2_score(dataG['violation_probability'], dataG['prediction'])
print("Cross-Predicted Accuracy:", accuracy)

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (24, 8)
plt.plot(dataG['prediction'][:100], label = "Pred")  # Load the 500 data points from prediction with label name 'Pred'
plt.plot(dataG['violation_probability'][:100], label = "Actual")